In [6]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
from sentence_transformers import SentenceTransformer, util

# load your own model
model = torch.load('apple_resnet_classifier.pt',  map_location=torch.device('cpu'))
model.to(device)
model.eval()

# load dummy sample images of 50
print('Enter folder location:')
folder_url = input()

transform_img_normal = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

dataset = ImageFolder(folder_url, transform=transform_img_normal)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=50, shuffle=False)




Enter folder location:


In [8]:
# calculate with model
def predict(mydata):

    # define the model labels and keep track of counts
    class_labels = ['Bad Apple', 'Normal Apple', 'Rot Apple', 'Scab Apple']
    class_counts = [0, 0, 0, 0]

# lets run the model over the
    with torch.no_grad():
        for data in mydata:
                inputs, labels = data

                # because weights are cuda casted need to be equal
                inputs = inputs.to(device)

                out = model(inputs).to(device)
                _, predicted = torch.max(out.data, 1)
                for p in predicted:
                    if p.item() == 0:
                        class_counts[0] += 1
                    elif p.item() == 1:
                        class_counts[1] += 1
                    elif p.item() == 2:
                        class_counts[2] += 1
                    else:
                        class_counts[3] += 1

                label_counts_dict = {label: count for label,
                                count in zip(class_labels, class_counts)}
        return label_counts_dict

In [10]:
predict(dataset_loader)
x = predict(dataset_loader)

In [11]:
mylist = []
mylist.append(x)

In [12]:
mylist

[{'Bad Apple': 5, 'Normal Apple': 32, 'Rot Apple': 6, 'Scab Apple': 7}]

In [ ]:
modelchat = <insert deleted model>

query_embedding = modelchat.encode(input('What is your question?'))
passage_embedding = modelchat.encode(mylist)



x = util.dot_score(query_embedding, passage_embedding)
print("Similarity:", x[[0]])

In [ ]:
print(x[0])

In [ ]:
import torch

torch.softmax(x[0], dim=-1)

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)
print(res)
# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

